In [0]:
# general setup
!apt-get update
!pip install selenium
!pip install selenium-wire
!pip install opencv-python
!pip install tqdm
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from seleniumwire import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
# If you get the old style of captcha just run again

In [0]:
import numpy as np
import cv2
import os
from PIL import Image
from io import StringIO
from io import BytesIO
import json
import time
import requests
from matplotlib import pyplot as plt
from tqdm import tqdm


In [0]:
from selenium.webdriver.common.by import By

def get_request():
  # Function to visit yandex.com and get a post request we can use to request captcha's
  
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get("https://passport.yandex.com")
  
  register_element=wd.find_element_by_css_selector(".control.link.link_theme_normal.passp-auth-header-link.passp-auth-header-link_visible")
  register_element.click()
  
  time.sleep(2) # Wait to make sure the no_phone_element is loaded
  no_phone_element=wd.find_element_by_css_selector(".toggle-link.link_has-no-phone")
  no_phone_element.click()
  
  for request in wd.requests:
      if request.response:
        if request.path=='https://passport.yandex.com/registration-validations/textcaptcha': # Find the right post request
          headers=request.headers
          data=request.body
          return headers,data

In [0]:

headers_get = {
      'Referer': 'https://passport.yandex.com/registration?from=mail&origin=hostroot_homer_auth_L_com&retpath=https%3A%2F%2Fmail.yandex.com%2F&process_uuid=af0fbb02-8dcc-4820-893b-28bb15fec84a',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
  }

def get_image(headers_post,data_post):
  response = requests.post('https://passport.yandex.com/registration-validations/textcaptcha', headers=headers_post, data=data_post)
  r = json.loads(response.text)
  key=r['key']
  params_get = (
        ('key', key),
    )
  response = requests.get('https://ext.captcha.yandex.net/image', headers=headers_get, params=params_get)
  image = Image.open(BytesIO(response.content))
  return image


In [0]:
def split_image(img):
    # if ndim!=3 we got the old style of captcha
    if img.ndim==3:

      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # isolate background
      for x in range(img.shape[1]):
        for y in range(img.shape[0]):
          if (img[y,x]==255).any():
            gray[y,x]=0
          else:
            gray[y,x]=255
      # Find contours

      ret, thresh = cv2.threshold(gray, 127, 255, 0)
      _,contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
      contours=sorted(contours, key=cv2.contourArea)
      contours=contours[-2:]
      images=[]

      # Seperate image from background
      for cnt in contours:
          x,y,w,h = cv2.boundingRect(cnt)
          crop= img[y:y+h, x:x+w]
          images.append(cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY))

      return images
    # If captcha is the old style      
    else:
      raise Exception('Old style captcha')

In [0]:
# https://www.pyimagesearch.com/2017/11/27/image-hashing-opencv-python/

def dhash(image, hashSize=8):
	# resize the input image, adding a single column (width) so we
	# can compute the horizontal gradient
	resized = cv2.resize(image, (hashSize + 1, hashSize))
 
	# compute the (relative) horizontal gradient between adjacent
	# column pixels
	diff = resized[:, 1:] > resized[:, :-1]
 
	# convert the difference image to a hash
	return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [9]:
# !rm -r yandex_captcha
# !rm -r content
!git clone https://github.com/tommanis1/yandex_captcha
# !unzip yandex_captcha/images_with_hash.zip

Cloning into 'yandex_captcha'...
remote: Enumerating objects: 11681, done.
remote: Counting objects: 100% (11681/11681), done.
remote: Compressing objects: 100% (11679/11679), done.
remote: Total 24057 (delta 1), reused 11681 (delta 1), pack-reused 12376
Receiving objects: 100% (24057/24057), 195.53 MiB | 36.06 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [18]:
# !sudo apt install git
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name" 
!cd yandex_captcha &&git add -A && git commit --message="Add my-file" && git push
# !git commit --message="Add my-file"
# !git push


[master 4e89bb4d] Add my-file
 1 file changed, 294 insertions(+)
 create mode 100644 index.php
fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
hashes=[]
files=os.listdir('images_with_hash')
for filename in files:
  Hash=filename.split('.')[0]
  hashes.append(Hash)
print(str(len(hashes))+' images already hashed')

30681 images already hashed


In [0]:
# Code to get captcha's, check if the hash already exists and save the image
for i in range(2):
  headers,data=get_request()
  for j in tqdm(range(1000)):
    image=np.asarray(get_image(headers,data))
    images=split_image(image)
    for img in images:
      Hash=str(dhash(img))
      if Hash not in hashes:
        hashes.append(Hash)
        cv2.imwrite(f'images_with_hash/{Hash}.png',img)
        print(i,j)
      else:
        print(i,j,'already in hashes')

  0%|          | 1/1000 [00:01<30:20,  1.82s/it]

0 0 already in hashes
0 0 already in hashes


  0%|          | 2/1000 [00:03<29:17,  1.76s/it]

0 1
0 1 already in hashes


  0%|          | 3/1000 [00:05<28:42,  1.73s/it]

...

 46%|████▌     | 459/1000 [22:12<18:13,  2.02s/it]

1 458 already in hashes
1 458


 46%|████▌     | 460/1000 [22:13<16:52,  1.88s/it]

1 459 already in hashes
1 459


ConnectionError: ignored

In [0]:
#downlload images
!zip -r /content/file.zip /content/images_with_hash
from google.colab import files
files.download("/content/file.zip")

updating: content/images_with_hash/ (stored 0%)
updating: content/images_with_hash/10868249038292178124.png (deflated 2%)
updating: content/images_with_hash/3181486278190000168.png (deflated 1%)
updating: content/images_with_hash/7548677607908403556.png (stored 0%)
updating: content/images_with_hash/5032290108792527675.png (stored 0%)
updating: content/images_with_hash/4943288377350443665.png (stored 0%)
updating: content/images_with_hash/7483833353787618653.png (stored 0%)
updating: content/images_with_hash/1729509076058259524.png (stored 0%)
updating: content/images_with_hash/8012245455186692667.png (stored 0%)
updating: content/images_with_hash/14179210325664845005.png (deflated 1%)
updating: content/images_with_hash/3301851859285855126.png (stored 0%)
updating: content/images_with_hash/5062542603495416840.png (deflated 1%)
updating: content/images_with_hash/10577509481627165859.png (deflated 2%)
updating: content/images_with_hash/14657362088575799961.png (stored 0%)
updating: conte

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 46626, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
# code to see what percentage of captcha's we can solve with the current dataset

partial_match=0
match=0
no_match=0
sample_size=100
for i in range(1):
  headers,data=get_request()
  for j in tqdm(range(sample_size)):
    
    image=np.asarray(get_image(headers,data))
    images=split_image(image)
    h1,h2=str(dhash(images[0])),str(dhash(images[1]))
    if h1 in hashes and h2 in hashes:
      match+=1
      continue
    elif h1 in hashes or h2 in hashes:
      partial_match+=1
      continue
    else:
      no_match+=1
print(str(round(match/sample_size*100))+'% was a full match')
print(str(round(partial_match/sample_size*100))+'% was a partial match')
print(str(round(no_match/sample_size*100))+'% was not a match')

100%|██████████| 100/100 [01:54<00:00,  1.15s/it]

22% was a full match
78% was a partial match
0% was not a match
